In [1]:
!date

Fri Feb 19 11:21:41 PST 2016


In [2]:
%matplotlib inline

from __future__ import division, print_function

import theano
import numpy as np
import matplotlib.pyplot as plt

from theano import tensor, function, shared

In [3]:
N = 100000

In [4]:
# Variable definitions
a = 0.05 # Learning rate
X = tensor.matrix('X') # Data locations
G = tensor.matrix('Y') # Grid point locations
M = shared(np.zeros(int(0.3 * N))) # Vector of priors
theta = shared(np.random.rand(int(0.3 * N))) # Latent Gaussian variables
y = tensor.vector('y') # Observations
scale = tensor.scalar('scale') # Distance scale in Mahalanobis distance
sigma_o = shared(20 * np.random.rand()) # Variance of data
sigma_p = shared(20 * np.random.rand()) # Variance of LGP

# Construct kernel weight matrix
K = tensor.sum(tensor.square(tensor.reshape(X, [-1, 1, 2]) - G) / (4 * tensor.square(scale)), -1)
norms = tensor.reshape(tensor.sum(K, axis=-1), [-1, 1])
row_norms = tensor.cast(tensor.eq(norms, tensor.zeros_like(norms)), 'float32') + norms
K_norm = K / norms

# Construct the likelihoods + priors
likelihood_obs = (-0.5 * tensor.log(2 * np.pi) -
                  tensor.log(sigma_o) -
                  tensor.sum(tensor.square(y - tensor.dot(K_norm, theta)) / (2 * tensor.square(sigma_o))))
param_prior = (-0.5 * tensor.log(2 * np.pi) -
                     tensor.log(sigma_p) -
                     tensor.sum(tensor.square(M - theta) / (2 * tensor.square(sigma_p))))
full_likelihood = -1 * (likelihood_obs + param_prior)

# Compute the gradients
gtheta, gsigma_o, gsigma_p = tensor.grad(full_likelihood, [theta, sigma_o, sigma_p])

# The MAP estimator
likelihood = function(inputs=[X, G, y, scale],
                      outputs=[full_likelihood, theta, sigma_o, sigma_p],
                      updates=[(theta, theta - a * gtheta),
                               (sigma_o, sigma_o - a * gsigma_o),
                               (sigma_p, sigma_p - a * gsigma_p),
                               (M, theta)])

In [ ]:
rs = np.random.RandomState(20160219)
X, G, y, scale = (100 * rs.rand(N, 2),
                  100 * rs.rand(int(0.3 * N), 2),
                  rs.randint(10, size=N), 2)

In [ ]:
%%time
batches = int(X.shape[0] / 1000) + 1
ll, so, sp = np.empty((100, batches)), np.empty((100, batches)), np.empty((100, batches))
t = np.empty((100, batches, int(0.3 * N)))

for epoch in range(100):
    for i, (mini_batch_x, mini_batch_y) in enumerate(zip(np.array_split(X, batches), np.array_split(y, batches))):
        ll[epoch, i], t[epoch, i, :], so[epoch, i], sp[epoch, i] = likelihood(mini_batch_x, G, mini_batch_y, scale)

In [ ]:
plt.hist(ll[np.where(np.isfinite(ll))])

In [ ]:
plt.hist(so[np.where(np.isfinite(so))])

In [ ]:
plt.hist(sp[np.where(np.isfinite(sp))])

In [ ]:
plt.hist(t[np.where(np.isfinite(t))])